In [12]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')


In [13]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [14]:
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [15]:
df['age'] = df['age'].fillna(df['age'].median())


In [16]:
df = df[['survived', 'sex', 'age', 'class', 'adult_male']]

In [17]:
df = pd.get_dummies(df, columns=['sex', 'class'], drop_first=True)


In [18]:
df.tail()

,survived,age,adult_male,sex_male,class_Second,class_Third
886,0,27.0,True,True,True,False
887,1,19.0,False,False,False,False
888,0,28.0,False,False,False,True
889,1,26.0,True,True,False,False
890,0,32.0,True,True,False,True


In [19]:
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = df.drop('survived', axis=1)
y = df['survived']

# Identify numeric columns
numeric_cols = ['age', 'adult_male']

# Initialize scaler
scaler = StandardScaler()

# Fit and transform numeric columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

X.head()


,age,adult_male,sex_male,class_Second,class_Third
0,-0.565736,0.811922,True,False,True
1,0.663861,-1.231645,False,False,False
2,-0.258337,-1.231645,False,False,True
3,0.433312,-1.231645,False,False,False
4,0.433312,0.811922,True,False,True


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7932960893854749

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.84      0.83       105
           1       0.76      0.73      0.74        74

    accuracy                           0.79       179
   macro avg       0.79      0.78      0.79       179
weighted avg       0.79      0.79      0.79       179



In [21]:
# Get feature names and coefficients
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

coef_df


,Feature,Coefficient
0,age,-0.057014
3,class_Second,-0.665810
2,sex_male,-0.719455
1,adult_male,-1.029920
4,class_Third,-1.911271


In [22]:
import numpy as np

coef_df['Odds_Ratio'] = np.exp(coef_df['Coefficient'])
coef_df


,Feature,Coefficient,Odds_Ratio
0,age,-0.057014,0.944580
3,class_Second,-0.665810,0.513857
2,sex_male,-0.719455,0.487017
1,adult_male,-1.029920,0.357036
4,class_Third,-1.911271,0.147892
